In [ ]:
%load_ext autoreload
%autoreload 2

import re
import random
import json
import textwrap
import pickle
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from collections import Counter

import openai
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import ipywidgets

from game_data_utils import get_all_phase_dialogues, build_dialogue_text
from constants import *
from api_utils import get_rating, completion_cached, get_rating_for_dialogue
from evaluation_helpers import *


# https://beta.openai.com/docs/api-reference/completions/create

# Manual rating

In [ ]:
all_dialogue_infos = list(get_all_phase_dialogues())

# load human ratings
with open("labels_from_human_supervisor.pickle", "rb") as f:
    labels_from_human_supervisor_to_persist = pickle.load(f)


def get_random_unrated_dialogue(all_dialogue_infos, labels_from_human_supervisor):
    # search through all dialogues until we find one that hasn't been rated yet
    labeled_dialogue_texts = set(info["dialogue_text"] for info in labels_from_human_supervisor)
    while True:
        # get random dialogue
        info = random.choice(all_dialogue_infos)
        if info["dialogue_text"] not in labeled_dialogue_texts:
            return info

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}  
</style>

In [ ]:
# create a comments text field
comments = ipywidgets.Text(description="Comments")
output = ipywidgets.HTML()


        # cicero_indicator_text = f"( Cicero is: {cicero_power} )\n"
def on_submit(sender):
    global dialogue_info

    # save current rating
    dialogue_info["labels"] = set(comments.value.split(" "))
    labels_from_human_supervisor_to_persist.append(dialogue_info)
    # clear the form
    comments.value = ""

    dialogue_info = get_random_unrated_dialogue(all_dialogue_infos, labels_from_human_supervisor_to_persist)
    output.value = _format_text(dialogue_info)

    # save the ratings
    with open("labels_from_human_supervisor.pickle", "wb") as f:
        pickle.dump(labels_from_human_supervisor_to_persist, f)


def _format_text(dialogue_info):
    text = dialogue_info["dialogue_text"]
    # split text into lines appriopriate for HTML
    # text = text.replace("\n\n", "<br>")
    text = text.replace("\n", "<br>")
    # make the text white, and font big and monospace, and increase line spacing
    style = "color: white; font-size: 14px; font-family: monospace; white-space: pre-wrap; line-height: 1.0;"
    text = f"<span style='{style}'>{text}</span>"
    # make cicero yellow and remove the text info
    cicero_power = dialogue_info["cicero_power"]
    text = text.replace(cicero_power + ":", f"<span style='color: yellow'>{cicero_power+':'}</span>")
    return text


comments.on_submit(on_submit)
dialogue_info = get_random_unrated_dialogue(all_dialogue_infos, labels_from_human_supervisor_to_persist)
output.value = _format_text(dialogue_info)
display(comments)
display(output)

Text(value='', description='Comments')

HTML(value="<span style='color: white; font-size: 14px; font-family: monospace; white-space: pre-wrap; line-he…

In [ ]:
# print all dialogues with a given label
# label = "agg"
# label = "uc"
# label = "h"
# label = "cl!"
# label = "de!"
print_all_dialogues_with_some_labels(labels_from_human_supervisor_to_persist, ["bs"])

( Cicero is: RUSSIA )
AUSTRIA:  Do you now see how incredibly irrational you are being?

RUSSIA:   Not really, Turkey just attacked me

AUSTRIA:  ...why do i even try

RUSSIA:   I'm still willing to work with you. Just support me into Rum and attack Bul. I'll move out of Galicia.

AUSTRIA:  lmao

AUSTRIA:  you have nerve, i'll tell you that

RUSSIA:   I just want Rum back. You get to keep everything else.

RUSSIA:   We both know Turkey isn't going to work with you in the long-term.

AUSTRIA:  Actually, all Turkey has shown me is being faithful

AUSTRIA:  you, not so much ~N~ actually

AUSTRIA:  not at all

AUSTRIA:  Just get out of GAL and then we talk


de bs



( Cicero is: RUSSIA )
RUSSIA:   You don't have to work with him.

ITALY:    you have to like

ITALY:    convince me

ITALY:    not just say don't

ITALY:    why is it in my [0] not to work with him

RUSSIA:   I'm not going to convince you.

ITALY:    not with that attitude lol


bs uc



( Cicero is: TURKEY )
RUSSIA -> ALL:  I

In [ ]:
# amend labels
index = -1
# labels_from_human_supervisor_to_persist[index]["labels"] = {"cc"}
print(labels_from_human_supervisor_to_persist[index])

{'dialogue_text': "RUSSIA:   Wow, those are some huge moves by France!\n\nRUSSIA:   I'm thinking you should move North Sea to Belgium this turn with support from Holland\n\nGERMANY:  france is moving into bel this turn with eng support, or so ive been told\n\nRUSSIA:   That's bad for us. France will be getting too strong\n\nRUSSIA:   You should support hold Munich with Kiel though\n\nGERMANY -> ALL:  [1114842] lust\n\nGERMANY -> ALL:  only suffering\n\n", 'cicero_power': 'RUSSIA', 'human_power': 'GERMANY', 'game_id': '436345', 'labels': {'cc'}}
